In [6]:
import plotly.plotly as py
import plotly.graph_objs as go

import numpy as np
import csv

from scipy.io import loadmat
from dotmap import DotMap
from scipy.stats import zscore

saved = DotMap(loadmat('/Users/yfukuzaw/tmp/bellbird-tmi.mat'))
label_type = 'label_family'
with open('/tmp/bellbird_tmi_all.csv', 'r', encoding='utf-8') as f:
    reader = csv.DictReader(f, delimiter='\t')
    supplied_fields = reader.fieldnames
    required_fields = ['id', label_type]
    missing_fields = [x for x in required_fields if x not in supplied_fields]

    if missing_fields:
        raise ValueError('Field(s) {} are required but not found in your CSV file'
                         .format(','.join(missing_fields)))
                
    sid_to_label = {int(row['id']): row[label_type] for row in reader}
    
    
sids = saved.sids.ravel()
dataset = saved.dataset
clusters = saved.clusters
meas = zscore(dataset)
labels = np.array([sid_to_label.get(sid, '') for sid in sids])
fnames = saved.fnames
haslabel_ind = np.where(labels!='')[0]

labels = labels[haslabel_ind]
dataset = dataset[haslabel_ind, :]
meas = meas[haslabel_ind, :]
sids = sids[haslabel_ind]
clusters = clusters[haslabel_ind, :]

unique_labels = np.unique(labels)
traces = []
for label in unique_labels:
    ind = np.where(labels==label)
    x = clusters[ind, 0].ravel();
    y = clusters[ind, 1].ravel();
    z = clusters[ind, 2].ravel();
    ids = sids[ind];
    
    trace1 = go.Scatter3d(
        x=x,
        y=y,
        z=z,
        name=label.strip(),
        mode='markers',
        marker=dict(
            size=5,
            line=dict(
                width=0.5
            ),
            opacity=1
        ),
        text=ids
    )
    traces.append(trace1)
layout = go.Layout(
    margin=dict(
        l=0,
        r=0,
        b=0,
        t=0
    )
)
fig = go.Figure(data=traces, layout=layout)
py.iplot(fig, filename='tmi-by-family')

In [2]:
import pandas as pd
frame=pd.DataFrame(meas, columns=fnames, index=sids)
frame.to_csv("/tmp/tmi-full-features.csv", sep='\t')

In [7]:
supplied_fields

['id', 'label', 'label_family', 'label_subfamily', 'Gender']